In [1]:
# Advent of Code, Beacon Scanner Day 19
from rich.jupyter import print as rprint
import numpy as np

DEBUG = False
MATCH = 12

# return a list of lists, each sublist containing beacon points relative to the scanner
def parse(input_file):
    lines = [
        [[tuple(int(x) for x in l.split(","))] for l in l[1:]]
        for l in [
            l.strip().split("\n")
            for l in open(input_file).read().split("\n\n")
        ]
    ]
    return lines

def distance(x, y):
    return np.linalg.norm(np.array(x) - np.array(y))

def inverse(matrix):
    return np.linalg.inv(matrix).astype(int)

def add(x, y):
    return tuple(np.array(x) + np.array(y))

def negative(x):
    return tuple(-1 * np.array(x))

def first_beacon(lines):
    beacon_1 = []
    for scanner in lines:
        for beacon_1 in scanner:
            beacon_1.append({distance(beacon_1[0], _[0]) for _ in scanner})

def create_beaconmap(lines, debug=DEBUG):
    beaconcount = len(lines)
    beacon_map = {}
    for i in range(beaconcount):
        for j in range(i+1, beaconcount):
            for x in lines[i]:
                if debug:
                    rprint("CB: [%d,%d]" % (i,j), x)
                for y in lines[j]:
                    if len(x[1] & y[1]) >= MATCH:
                        if debug:
                            print("match",x[1] & y[1])
                        beacon_map.setdefault((i, j), set()).add((x[0], y[0]))
    return beacon_map

def generate_orientations(beaconmap, orientations, debug=DEBUG):
    transforms = {}
    for b,vals in beaconmap.items():
        if DEBUG:
            rprint("GO:", b, vals)
        for orientation in orientations:
            s = set()
            for val in vals:
                s.add(distance(val[0], np.dot(val[1],orientation)))
            if debug:
                rprint("GO: Result set:",s)
            if len(s) == 1:
                X, Y = val[0], np.dot(val[1],orientation)
                transforms[b] = [(add(X,negative(Y)), orientation)]
                break
    return transforms

def apply_transforms(lines, transforms, debug=DEBUG):
    beaconcount = len(lines)
    positions = []
    points = {p[0] for p in lines[0]}

    for i in range(1, beaconcount):
        for p in lines[i]:
            testpoint = (0,0,0)
            point = p[0]
            for trans in transforms[(0,i)]:
                testpoint = add(trans[0], np.dot(testpoint, trans[1]))
                point = add(trans[0], np.dot(point, trans[1]))
                if debug:
                    rprint(i, p, trans[0], point)
            points.add(point)
        positions.append(testpoint)
        if debug:
            rprint("AT:", i, positions)
    if debug:
        rprint("AT:", points, positions)
    return(points, positions)

In [2]:
def iterate_over_beacons(t, beaconcount):
    # relative to the first beacon
    while len([*filter(lambda x: x[0] == 0, t)]) < beaconcount - 1:
        for i in range(beaconcount):
            for j in range(beaconcount):
                for k in range(beaconcount):
                    # don't do the same point
                    if j == k or i == k:
                        continue
                    if (
                        (relative_1 := t.get((i, j)))
                        and not (i, k) in t
                        and (relative_2 := t.get((j, k)))
                    ):
                        t[(i, k)] = [*relative_2, *relative_1]
                    if (
                        (relative_1 := t.get((i, j)))
                        and not (i, k) in t
                        and (relative_2 := t.get((k, j)))
                    ):
                        t[(i, k)] = [
                        *[
                            (np.dot(negative(v[0]), inv := inverse(v[1])), inv)
                            for v in reversed(relative_2)
                        ],
                        *relative_1,
                    ]
    return t

In [3]:
def scan_o_matic(input_file):
    lines = parse(input_file)
    beaconcount = len(lines)
    
    beacon_1 = first_beacon(lines)
    beaconmap = create_beaconmap(lines)
    
    transforms = generate_orientations(beaconmap, orientations)
    transforms = iterate_over_beacons(transforms, beaconcount)
    points, positions = apply_transforms(lines, transforms)
    return positions, points

In [4]:
# x, y, z
directions =(( 1, 0, 0), ( 0, 1, 0), ( 0, 0, 1), (-1, 0, 0), ( 0,-1, 0), ( 0, 0,-1))

orientations = [
    (x, y, z)
    for x in directions
    for y in directions
    for z in directions
    if np.linalg.det((x, y, z)) == 1
]

In [5]:
positions, points = scan_o_matic("input_files/day19.test.txt")
# part 1: 79
rprint(len(points))
# Part 2: 3621
rprint(np.max([np.sum(np.abs(add(i,negative(j)))) for i in positions for j in positions]))

79

3621

In [6]:
positions, points = scan_o_matic("input_files/day19.txt")
# part 1: 318
rprint(len(points))
# Part 2: 12166
rprint(np.max([np.sum(np.abs(add(i,negative(j)))) for i in positions for j in positions]))

318

12166